In [2]:
import sys
import pathlib
import sys
import glob
import duckdb
import pandas
from tabula.io import read_pdf
import tabula
import os
import json
from tqdm import tqdm

In [6]:
save_path = '/N/project/TIC/Humana/99213_ffs_negotiated'

In [16]:
new_files = pandas.read_csv("/N/project/cryptocurrency_data/TIC_DATA/Chunkify/Downloaded_Files_1.csv", dtype={'File_Name': 'str'})
new_files=list(new_files.File_Name)

In [57]:
cols = ["NPI", "NEGOTIATION_ARRANGEMENT","BILLING_CLASS", "BILLING_CODE", "NEGOTIATED_RATE","NEGOTIATED_TYPE", "TIN", "TYPE", "BILLING_CODE_TYPE"]
bl_code = ['99213'] 

In [61]:
new_files[0] = '/N/project/TIC/Humana/data_10_2022/00000.csv.gz'
pandas.read_csv(new_files[0], sep = '~', dtype=str).head()

,REPORTING_ENTITY_NAME,REPORTING_ENTITY_TYPE,LAST_UPDATED_ON,VERSION,NPI,TIN,TYPE,NEGOTIATION_ARRANGEMENT,NAME,BILLING_CODE_TYPE,...,NEGOTIATED_RATE,EXPIRATION_DATE,SERVICE_CODE,BILLING_CLASS,BILLING_CODE_MODIFIER,ADDITIONAL_INFO,BUNDLED_BILLING_CODE_TYPE,BUNDLED_BILLING_CODE_VERSION,BUNDLED_BILLING_CODE,BUNDLED_DESCRIPTION
0,Humana Inc,Health Insurance Issuer,2022-10-25,1.0.0,1548357973,370661199,ein,ffs,Durable Medical Equipment (Other than renal)-R...,RC,...,0,9999-12-31,NaN,institutional,NaN,NaN,NaN,NaN,NaN,NaN
1,Humana Inc,Health Insurance Issuer,2022-10-25,1.0.0,1548357973,370661199,ein,ffs,Durable Medical Equipment (Other than renal)-O...,RC,...,0,9999-12-31,NaN,institutional,NaN,NaN,NaN,NaN,NaN,NaN
2,Humana Inc,Health Insurance Issuer,2022-10-25,1.0.0,1649259656,481202402,ein,ffs,Intracranial Vascular Procedures with Principa...,MS_DRG,...,4069,9999-12-31,NaN,institutional,NaN,"DAYS: 001-999 ,ALLOWED AMOUNT: 4069,REV: 110,1...",NaN,NaN,NaN,NaN
3,Humana Inc,Health Insurance Issuer,2022-10-25,1.0.0,1649259656,481202402,ein,ffs,Intracranial Vascular Procedures with Principa...,MS_DRG,...,4069,9999-12-31,NaN,institutional,NaN,"DAYS: 001-999 ,ALLOWED AMOUNT: 4069,REV: 110,1...",NaN,NaN,NaN,NaN
4,Humana Inc,Health Insurance Issuer,2022-10-25,1.0.0,1649259656,481202402,ein,ffs,Craniotomy with Major Device Implant or Acute ...,MS_DRG,...,4069,9999-12-31,NaN,institutional,NaN,"DAYS: 001-999 ,ALLOWED AMOUNT: 4069,REV: 110,1...",NaN,NaN,NaN,NaN


In [56]:
odata = pandas.read_csv(new_files[0], sep = '~', dtype=str).head()
odata.columns

Index(['REPORTING_ENTITY_NAME', 'REPORTING_ENTITY_TYPE', 'LAST_UPDATED_ON',
       'VERSION', 'NPI', 'TIN', 'TYPE', 'NEGOTIATION_ARRANGEMENT', 'NAME',
       'BILLING_CODE_TYPE', 'BILLING_CODE_TYPE_VERSION', 'BILLING_CODE',
       'DESCRIPTION', 'NEGOTIATED_TYPE', 'NEGOTIATED_RATE', 'EXPIRATION_DATE',
       'SERVICE_CODE', 'BILLING_CLASS', 'BILLING_CODE_MODIFIER',
       'ADDITIONAL_INFO', 'BUNDLED_BILLING_CODE_TYPE',
       'BUNDLED_BILLING_CODE_VERSION', 'BUNDLED_BILLING_CODE',
       'BUNDLED_DESCRIPTION'],
      dtype='object')

In [1]:
import pathlib
import pandas
import os
import json

def read_content(p, cols, codes):
    """
    Parse Humana in-network file for specific billing codes.

    Args:
        p: pathlib.Path object for humana file
        cols: list of columns to extract from the file
        codes: list of billing codes to search for

    Returns:
        pandas.DataFrame with relevant rows and columns from the file.
        If the file is empty (<0.5MB) or malformed, return None.

    """

    def has_content(p):
        """File > 0.5 MB"""
        return p.stat().st_size / 1e6 > 0.5

    df = None
    if has_content(p):
        try:
            df = (
                pandas.read_csv(p, sep = '~', dtype=str, usecols=cols)
                .query("BILLING_CODE.isin(@codes) & NEGOTIATION_ARRANGEMENT=='ffs' & NEGOTIATED_TYPE=='negotiated'")
                .assign(NPI=lambda df: df.NPI.str.split(","))
                .explode("NPI", ignore_index=True)
            )
        except EOFError:
            df = None
    return df

In [4]:
file_name = '02222.csv.gz' #CSV GZ File name
cols = ["NPI", "NEGOTIATION_ARRANGEMENT","BILLING_CLASS", "BILLING_CODE", "NEGOTIATED_RATE","NEGOTIATED_TYPE", "TIN", "TYPE", "BILLING_CODE_TYPE"]
bl_code = ['99213'] # Billing Codes ['99213', '232134']
infile= '/N/project/TIC/Humana/data_10_2022/' + file_name
infile = pathlib.Path(infile) #sys.argv[1])
df_humana = read_content(infile, cols, bl_code)
print("The output from the CSV GZ File:", df_humana)
save_path = '/N/project/TIC/Humana/99213_ffs_negotiated' #Save File Path
csv_file_name = 'sample_csv.csv' #File Name
df_humana.to_csv(save_path + '/' + csv_file_name, index=False, sep=",") #Export CSV File

The output from the CSV GZ File:             NPI        TIN TYPE NEGOTIATION_ARRANGEMENT BILLING_CODE_TYPE  \
0    1841353174  205432275  ein                     ffs               CPT   
1    1972576320  010594994  ein                     ffs               CPT   
2    1760634505  010594994  ein                     ffs               CPT   
3    1528049293  741996568  ein                     ffs               CPT   
4    1225020076  271320706  ein                     ffs               CPT   
..          ...        ...  ...                     ...               ...   
135  1154483089  310833261  ein                     ffs               CPT   
136  1477548758  142686322  npi                     ffs               CPT   
137  1124076468  297589507  npi                     ffs               CPT   
138  1831382423  340714585  npi                     ffs               CPT   
139  1982694329  390807236  npi                     ffs               CPT   

    BILLING_CODE NEGOTIATED_TYPE NEGOTIATE

,NPI,TIN,TYPE,NEGOTIATION_ARRANGEMENT,BILLING_CODE_TYPE,BILLING_CODE,NEGOTIATED_TYPE,NEGOTIATED_RATE,BILLING_CLASS


In [59]:
save_path = '/N/project/TIC/Humana/99213_ffs_negotiated'
df_humana.to_csv(save_path + '/' + 'Filter_2022-10-26_1_in-network-rates_000000000000.csv', index=False, sep=",")

In [50]:
def read_content(p, cols, codes):
    """
    Parse Humana in-network file for specific billing codes.

    Args:
        p: pathlib.Path object for humana file
        cols: list of columns to extract from the file
        codes: list of billing codes to search for

    Returns:
        pandas.DataFrame with relevant rows and columns from the file.
        If the file is empty (<0.5MB) or malformed, return None.

    """

    def has_content(p):
        """File > 0.5 MB"""
        return p.stat().st_size / 1e6 > 0.5

    df = None
    if has_content(p):
        try:
            df = (
                pandas.read_csv(p, sep = '~', dtype=str, usecols=cols)
                .query("BILLING_CODE.isin(@codes) & NEGOTIATION_ARRANGEMENT=='ffs' & NEGOTIATED_TYPE=='negotiated'")
                .assign(NPI=lambda df: df.NPI.str.split(","))
                .explode("NPI", ignore_index=True)
            )
        except EOFError:
            df = None
    return df

In [49]:
df = (
    pandas.read_csv(new_files[0], sep = '~', dtype=str, usecols=cols)
    .query("BILLING_CODE.isin(@bl_code)")
    .assign(NPI=lambda df: df.NPI.str.split(","))
    .explode("NPI", ignore_index=True)
)

In [51]:
for file in tqdm((new_files)):
    
    infile = pathlib.Path(file) #sys.argv[1])
    df_humana = read_content(infile, cols, bl_code) #df_cms.code.to_list())
    
    df_humana.to_csv(save_path + '/' + file.rsplit('/',1)[1], index=False, sep=",")

100%|██████████| 50/50 [06:22<00:00,  7.64s/it]


In [10]:
import glob
from tqdm import tqdm 

In [11]:
path = "/N/project/TIC/Humana/99213_ffs_negotiated"
csv_files = glob.glob(os.path.join(path, "*.csv.gz"))

df1 = None

for files in tqdm(csv_files):
    temp = pandas.read_csv(files)
    if df1 is None:
        df1 = temp
    else:
        df1 = pandas.concat([temp,df1],axis = 0)

100%|██████████| 212/212 [00:05<00:00, 37.69it/s]


In [12]:
df1.to_csv("/N/project/TIC/Humana/99213_ffs_negotiated/consolidated_99213_ffs.csv", index=False)